In [1]:
import os
import pandas as pd
import numpy as np
import sys
import csv
from time import strptime
from datetime import datetime as dt

In [221]:
csv.field_size_limit(sys.maxsize)
np.set_printoptions(threshold=sys.maxsize)

In [222]:
names_full = pd.read_csv('data/numbers_budget.csv')

In [223]:
names_full.shape

(5928, 5)

In [224]:
tmdb = pd.read_csv('data/tmdb/tmdb_5000_movies.csv')

In [225]:
full_data = pd.read_csv('data/tmdb/fulltmdb.csv')

In [226]:
def format_date(date):
    try:
        date_obj = dt.strptime(date, '%Y-%m-%d')
        return dt.strftime(date_obj, '%b %d, %Y')
    except:
        return 'False'

def has_day_month_year(date):
    date = date.replace(u'\xa0', u' ')
    return len(date.split(' ')) == 3

def format_tmdb_date(row):
    date = row.release_date.to_string(index=False).strip()
    return format_date(date) if date != 'NaN' else 'False'

def get_tmdb_date(name):
    item = tmdb.loc[tmdb['original_title'] == name]
    if item.shape[0]:
        return format_tmdb_date(item)
    item = full_data.loc[full_data['original_title'] == name]
    if item.shape[0]:
        return format_tmdb_date(item)

def replace_date(row):
    date = row['ReleaseDate']
    if not has_day_month_year(date):
        row['ReleaseDate'] = get_tmdb_date(row['Movie'])
    return row

In [227]:
added_dates = names_full.apply(replace_date, axis=1)

In [228]:
added_dates.shape

(5928, 5)

In [229]:
# movies without dates, skip them
added_dates[(added_dates['ReleaseDate'] == 'False') | (pd.isna(added_dates['ReleaseDate']))].shape

(49, 5)

In [230]:
with_dates = added_dates[(added_dates['ReleaseDate'] != 'False') & (~pd.isna(added_dates['ReleaseDate']))]

In [231]:
with_dates.shape

(5879, 5)

In [232]:
zero_worldwide_gross = with_dates.loc[with_dates['WorldwideGross'] == 0]

In [233]:
zero_budget = with_dates.loc[with_dates['ProductionBudget'] == 0]

In [234]:
gross_movies = with_dates.loc[(with_dates.WorldwideGross > 0) & (with_dates.ProductionBudget > 0)]

In [235]:
zero_worldwide_gross.shape, zero_budget.shape, gross_movies.shape

((348, 5), (0, 5), (5531, 5))

In [238]:
gross_movies.columns

Index(['ReleaseDate', 'Movie', 'ProductionBudget', 'DomesticGross',
       'WorldwideGross'],
      dtype='object')

In [244]:
def get_month(date):
    return strptime(date.split(' ')[0], '%b').tm_mon

def get_day(date):
    return date.split(' ')[1].split(',')[0]

def get_year(date):
    return date.split(' ')[2]

def date_if_weekend(date):
    return str(dt.strptime(date, '%b %d, %Y').weekday() > 4)

def apply_func_on_dataframe(df, col, func_list):
    return [[func(row[col]) for i, row in df.iterrows()] for func in func_list]

def get_date_lists(df):
    return apply_func_on_dataframe(df, 'ReleaseDate', [get_year, get_month, get_day, date_if_weekend])

def calculate_profitability(revenue, budget):
    revenue, budget = float(revenue), float(budget)
    profit = revenue - budget
    return profit/budget if profit > 0 else profit/revenue 

def get_profitability_df(df, col_revenue, col_budget):
    return [calculate_profitability(row[col_revenue], row[col_budget]) for i, row in df.iterrows()]

In [245]:
list_profitability = get_profitability_df(gross_movies, 'WorldwideGross', 'ProductionBudget')

In [246]:
df = gross_movies.copy()
df['profitability'] = list_profitability
df = df.drop('DomesticGross', axis=1)
df

,ReleaseDate,Movie,ProductionBudget,WorldwideGross,profitability
0,"Apr 23, 2019",Avengers: Endgame,400000000,2797800564,5.9945014100
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,379000000,1045663875,1.7590075858
2,"Apr 22, 2015",Avengers: Age of Ultron,365000000,1403013963,2.8438738712
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,306000000,2068223624,5.7589007320
4,"Apr 25, 2018",Avengers: Infinity War,300000000,2048359754,5.8278658467
...,...,...,...,...,...
5920,"Oct 8, 2004",Primer,7000,841926,119.2751428571
5921,"May 26, 2006",Cavite,7000,71644,9.2348571429
5924,"Apr 2, 1999",Following,6000,240495,39.0825000000
5925,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,-2.7369207773


In [247]:
deleted_least_profitable = df.sort_values('profitability')[50:-1]

In [248]:
list_year, list_month, list_day, list_if_weekend = get_date_lists(deleted_least_profitable)

In [249]:
deleted_least_profitable['year'] = list_year
deleted_least_profitable['month'] = list_month
deleted_least_profitable['day'] = list_day
deleted_least_profitable['weekend'] = list_if_weekend
deleted_least_profitable = deleted_least_profitable.drop('ReleaseDate', axis=1)

In [250]:
deleted_least_profitable

,Movie,ProductionBudget,WorldwideGross,profitability,year,month,day,weekend
4692,Hevi reissu,3800000,10153,-373.2736137102,2018,10,5,False
5098,Say Uncle,2000000,5361,-372.0647267301,2006,6,23,False
4745,Taxman,3500000,9871,-353.5740046601,1999,9,17,False
4674,The Loved Ones,4000000,12302,-324.1503820517,2012,6,1,False
4519,All Hat,4900000,15198,-321.4108435320,2008,1,18,False
...,...,...,...,...,...,...,...,...
5851,Super Size Me,65000,22233808,341.0585846154,2004,5,7,False
5542,The Blair Witch Project,600000,248300000,412.8333333333,1999,7,14,False
5820,The Gallows,100000,41656474,415.5647400000,2015,7,9,False
5629,Paranormal Activity,450000,194183034,430.5178533333,2009,9,25,False


In [252]:
deleted_least_profitable.profitability.quantile(np.linspace(.25, 1, 3, 0))

0.2500000000   -0.3783815046
0.5000000000    0.9208483357
0.7500000000    3.0160915000
Name: profitability, dtype: float64

In [254]:
deleted_least_profitable.shape

(5480, 8)

In [253]:
deleted_least_profitable.to_csv('data/cleaned_the_numbers.csv', index=False)